In [1]:
import requests
from bs4 import BeautifulSoup
import random
#import time
#import re
import pandas as pd

In [2]:
headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
class Proxy():
    proxy_url = 'https://www.socks-proxy.net/'
    proxy_list = []
    headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 
               'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

    def __init__(self):
        self.current_proxies = {}
        page = requests.get(self.proxy_url, headers=self.headers)
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'lxml')
            trs_list = soup.html.body.find('tbody').findAll('tr')
            for tr in trs_list:
                tds = tr.findAll('td')
                if tds[6].text.lower() == 'yes':
                    self.proxy_list.append((tds[4].text.lower(), f'{tds[0].text}:{tds[1].text}'))
        

    def change_proxy(self, num_tries=3):
        tries = 0
        while tries < num_tries:
            tries += 1
            proxy = random.choice(self.proxy_list)
            try:
                proxies={'http' : f'{proxy[0]}://{proxy[1]}',
                         'https' : f'{proxy[0]}://{proxy[1]}'}
                page = requests.get('https://hh.ru', headers=self.headers, proxies=proxies)
                if page.status_code == 200:
                    self.current_proxies = proxies
                    break
            except requests.exceptions.ConnectionError:
                continue
                
    @property
    def get_proxy(self):
        return self.current_proxies
    
    def get_proxy_list(self):
        return self.proxy_list

In [4]:
class CurrencyExchange():
    
    def __init__(self):
        self.usd_rate = 1
        self.eur_rate = 1
        self.update_rates_online()
        
    def update_rates_online(self):
        cbr_page = requests.get('http://www.cbr.ru', headers=headers)
        if cbr_page.status_code == 200:
            soup = BeautifulSoup(cbr_page.text, 'lxml')
            table = soup.findAll('tbody')[2].findAll('tr')
            rates = []
            for tr in table[1:]:
                trs = tr.findAll('td')
                rates.append(float(trs[1].text.strip().split()[1].replace(',','.')))
            self.usd_rate, self.eur_rate = rates
            
    @property
    def usd(self):
        return self.usd_rate
    
    @property
    def eur(self):
        return self.eur_rate

In [5]:
proxy = Proxy()
if not proxy.get_proxy:
    proxy.change_proxy()
proxy.get_proxy

{'http': 'socks4://209.203.130.25:54321',
 'https': 'socks4://209.203.130.25:54321'}

In [6]:
def proxy_request(url, headers, proxy=Proxy(), num_tries=3):
    if not proxy.get_proxy:
        proxy.change_proxy()
    proxies = proxy.get_proxy
    
    tries = 0
    while tries < num_tries:
        tries += 1
        try:
            page = requests.get(url, headers=headers, proxies=proxies)
            return page
        except requests.exceptions.ConnectionError:
            proxy.change_proxy()
            proxies = proxy.get_proxy
    return ''

In [7]:
#proxy.change_proxy()
#page = proxy_request('https://yandex.ru/internet', headers, proxy)
#if page.status_code == 200:
#    print('OK')
#    soup = BeautifulSoup(page.text, 'lxml')
#    current_ip = soup.find('li').findAll('div')[1].text
#    print(f'Using proxy {current_ip}')

In [8]:
def get_hh_jobs(job_string, proxy, pages=1):
    
    def make_url(job_string, page_num=0):
        BASE_HH_URL = 'https://hh.ru/search/vacancy?area=1&st=searchVacancy&text='
        url = ''
        if page_num:
            url = BASE_HH_URL + '+'.join(job_string.strip().split()) + '&page=' + str(page_num)
        else:
            url = BASE_HH_URL + '+'.join(job_string.strip().split())
        return url
    
    print(f'Collecting {pages} pages from HeadHunter')   
    vacancies = []
    page_num = 0
    
    page = proxy_request(make_url(job_string), headers, proxy)
    
    while (page.status_code == 200) and (page_num < pages):
        soup = BeautifulSoup(page.text, 'lxml')
        try:
            tmp_list = soup.find('div', attrs = {'class' : 'vacancy-serp'}).findAll('div', attrs = {'class' : 'vacancy-serp-item'})
            vacancies.extend(tmp_list)
        except AttributeError:
            pass
        page_num += 1
        page = proxy_request(make_url(job_string, page_num), headers, proxy)
    
    return vacancies

In [9]:
def get_superjob_jobs(job_string, proxy, pages=1):
    
    def make_url(job_string, page_num=0):
        BASE_URL = r'https://www.superjob.ru/vacancy/search/?keywords={}&geo%5Bt%5D%5B0%5D=4'
        url = ''
        if page_num:
            url = BASE_URL.format('%20'.join(job_string.strip().split())) + '&page=' + str(page_num + 1)
        else:
            url = BASE_URL.format('%20'.join(job_string.strip().split()))
        return url
    
    print(f'Collecting {pages} pages from SuperJob')   
    vacancies = []
    page_num = 0
    
    page = proxy_request(make_url(job_string), headers, proxy)
    
    while (page.status_code == 200) and (page_num < pages):
        soup = BeautifulSoup(page.text, 'lxml')
        try:
            tmp_list = soup.findAll('div', attrs = {'class' : 'f-test-vacancy-item'})
            vacancies.extend(tmp_list)
        except AttributeError:
            pass
        page_num += 1
        page = proxy_request(make_url(job_string, page_num), headers, proxy)
    
    return vacancies

In [10]:
def parce_hh_item(item):
    
    def get_compensation(compensation):
        
        min_salary = None
        max_salary = None
        currency = None
        
        temp = compensation.text.replace('\xa0', '').split()
        if len(temp) == 2:
            min_salary, max_salary = temp[0].split('-')
            currency = temp[1]
        elif temp[0] == 'от':
            min_salary = temp[1]
            currency = temp[2]
        elif temp[0] == 'до':
            max_salary = temp[1]
            currency = temp[2]
        return min_salary, max_salary, currency
    
    item_dict = {}
    
    item_dict['source'] = 'HeadHunter'  
    item_dict['premium'] = 'vacancy-serp-item_premium' in item.get('class')
    item_dict['title'] = item.find('a', attrs = {'data-qa' : 'vacancy-serp__vacancy-title'}).text
    item_dict['vacancy_link'] = item.find('a', attrs = {'data-qa' : 'vacancy-serp__vacancy-title'}).get('href')
    item_dict['employer'] = item.find('a', attrs = {'data-qa' : 'vacancy-serp__vacancy-employer'}).text.strip()
    item_dict['employer_link'] = 'https://hh.ru' + \
        item.find('a', attrs = {'data-qa' : 'vacancy-serp__vacancy-employer'}).get('href')
    item_dict['location'] = item.find('span', attrs = {'data-qa' : 'vacancy-serp__vacancy-address'}).text
    item_dict['responsibility'] = item.find('div', attrs = {'data-qa' : 'vacancy-serp__vacancy_snippet_responsibility'}).text
    item_dict['requirement'] = item.find('div', attrs = {'data-qa' : 'vacancy-serp__vacancy_snippet_requirement'}).text
    item_dict['date'] = item.find('span', attrs = {'data-qa' : 'vacancy-serp__vacancy-date'}).text.replace('\xa0', ' ')
    
    compensation = item.find('div', attrs = {'class' : 'vacancy-serp-item__compensation'})
    if compensation:
        item_dict['min_salary'], item_dict['max_salary'], item_dict['currency'] = get_compensation(compensation)
    
    return item_dict

In [11]:
def parce_superjob_item(item):
    
    def get_compensation(compensation):
        
        currency_dict = {
                36 : 'USD',
                8364 : 'EUR',
                8381 : 'руб.'
            }
        
        min_salary = None
        max_salary = None
        currency = None
        
        if compensation != 'По договорённости':
            currency = currency_dict[ord(compensation[-1])]
            compensation = compensation[:-1]
            if '—' in compensation:
                min_salary, max_salary = compensation.split('—')            
            else:
                if compensation[:2] == 'от':
                    min_salary = compensation[2:]
                elif compensation[:2] == 'до':
                    max_salary = compensation[2:]
                else:
                    min_salary = max_salary = compensation
        return min_salary, max_salary, currency
    
    
    item_dict = {}
    
    item_dict['source'] = 'SuperJob'
    
    a_tags = item.findAll('a')
    item_dict['title'] = a_tags[-2].text
    item_dict['vacancy_link'] = a_tags[-2].get('href')
    item_dict['employer'] = a_tags[-1].text
    item_dict['employer_link'] = 'https://www.superjob.ru/' + a_tags[-1].get('href')
    span_tags = item.find('span', attrs = {'class' : 'f-test-text-company-item-location'}).findAll('span', recursive=False)
    item_dict['location'] = span_tags[1].text
    item_dict['date'] = span_tags[0].text
    
    description = item.findAll('span', attrs = {'class' : '_3mfro _9fXTd _2JVkc _2VHxz _15msI'})
    for desc_item in description:
        if desc_item.span.text == 'Должностные обязанности:':
            item_dict['responsibility'] = desc_item.text[24:].strip()
        if desc_item.span.text == 'Требования:':
            item_dict['requirement'] = desc_item.text[11:].strip()
    
    
    compensation = item.find('span', attrs = {'class' : 'f-test-text-company-item-salary'}).text.replace('\xa0', '')
    item_dict['min_salary'], item_dict['max_salary'], item_dict['currency'] = get_compensation(compensation)
    
    return item_dict

In [12]:
def get_vacancies_all(job_string, num_pages=1):
    
    proxy = Proxy()
    proxy.change_proxy()
    
    vacancies = []
    
    hh_jobs = get_hh_jobs(job_string, proxy, num_pages)
    vacancies.extend([parce_hh_item(item) for item in hh_jobs])
    
    sj_jobs = get_superjob_jobs(job_string, proxy, num_pages)
    vacancies.extend([parce_superjob_item(item) for item in sj_jobs])
    
    data = pd.DataFrame(vacancies, columns=['title', 'vacancy_link', 'employer', 'employer_link', 
                                            'source', 'premium', 'location', 'responsibility', 'requirement',
                                            'date', 'min_salary', 'max_salary', 'currency'])
    
    return data    

In [13]:
data = get_vacancies_all('data science')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 13 columns):
title             22 non-null object
vacancy_link      22 non-null object
employer          22 non-null object
employer_link     22 non-null object
source            22 non-null object
premium           20 non-null object
location          22 non-null object
responsibility    22 non-null object
requirement       22 non-null object
date              22 non-null object
min_salary        8 non-null object
max_salary        5 non-null object
currency          8 non-null object
dtypes: object(13)
memory usage: 2.3+ KB


In [14]:
data.head()

,title,vacancy_link,employer,employer_link,source,premium,location,responsibility,requirement,date,min_salary,max_salary,currency
0,Data Scientist,https://hh.ru/vacancy/35011897?query=data%20sc...,HeadHunter::Analytics/Data Science,https://hh.ru/employer/1455?dpt=hh-1455-ds,HeadHunter,True,"Москва, Алексеевская",Статистический анализ и обработка данных. Пост...,Опыт реализации алгоритмов машинного обучения ...,16 декабря,200000,None,руб.
1,Data Analyst,https://hh.ru/vacancy/35033890?query=data%20sc...,ЗАО ОКБ,https://hh.ru/employer/2129243,HeadHunter,True,"Москва, Третьяковская и еще 2",Построение и поддержка системы мониторинга и о...,Опыт работы аналитиком по данным в риск-подраз...,17 декабря,NaN,NaN,NaN
2,Data Analyst (Senior),https://hh.ru/vacancy/33864288?query=data%20sc...,Сбербанк для экспертов,https://hh.ru/employer/3529?dpt=3529-3529-prof,HeadHunter,True,Москва,Анализ бизнес требований и формулировка задачи...,Понимание математической статистики. Знание SQ...,13 декабря,NaN,NaN,NaN
3,Product Analyst,https://hh.ru/vacancy/32845591?query=data%20sc...,HeadHunter::Analytics/Data Science,https://hh.ru/employer/1455?dpt=hh-1455-ds,HeadHunter,True,"Москва, Алексеевская",1. Собирать и анализировать данные о поведении...,Опыт построения аналитических моделей и тестир...,16 декабря,NaN,NaN,NaN
4,Рекрутер (подбор d-people)/Team lead,https://hh.ru/vacancy/32426044?query=data%20sc...,Сбербанк для экспертов,https://hh.ru/employer/3529?dpt=3529-3529-prof,HeadHunter,True,"Москва, Кутузовская и еще 1","Закрытие вакансий в направлении Data Science, ...",Опыт работы рекрутером/ менеджером по подбору ...,13 декабря,NaN,NaN,NaN


In [15]:
data.currency.value_counts()

руб.    8
Name: currency, dtype: int64

In [16]:
cur_change = CurrencyExchange()

In [17]:
cur_change.usd, cur_change.eur

(62.5831, 69.6925)